In [1]:
import grpc

import tensorflow as tf

from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

In [2]:
from keras_image_helper import create_preprocessor

In [3]:
host = 'localhost:8500'

channel = grpc.insecure_channel(host)

stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

### VGG16

In [4]:
from tensorflow.keras.applications.vgg16 import preprocess_input

### Feature extraction that change image dimension shape to 5 x 5 x 512
conv_base = tf.keras.applications.vgg16.VGG16(
    weights="imagenet",
    include_top=False,
    input_shape=(180, 180, 3)
)

In [5]:
from tensorflow.keras.preprocessing.image import load_img

In [6]:
import os
import numpy as np

print(os.getcwd())

D:\github_repos\mlzoomcamp\MLZoomcamp_2022\10_Kubernetes_TF_serve


In [7]:
name = 'woodpecker_exampl.webp'
img = load_img(os.path.join(os.getcwd(), 'sequentials', name), target_size=(180, 180))

In [8]:
x = np.array(img)
x.shape

(180, 180, 3)

In [9]:
X = np.array([x])
X.shape

(1, 180, 180, 3)

In [ ]:
### It is okay to skip this step
X_preprocessed = preprocess_input(X)

In [10]:
pred = conv_base.predict(X)

1/1 [==============================] - 4s 4s/step


In [ ]:
### not used
preprocessor = create_preprocessor('vgg16', target_size=(180, 180))

In [11]:
def np_to_protobuf(data):
    return tf.make_tensor_proto(data, shape=data.shape)

In [12]:
pb_request = predict_pb2.PredictRequest()

#pb_request.model_spec.name = 'seq-model-dir'
pb_request.model_spec.name = 'sequential'
pb_request.model_spec.signature_name = 'serving_default'

pb_request.inputs['input_19'].CopyFrom(np_to_protobuf(pred))

In [13]:
pb_request

model_spec {
  name: "sequential"
  signature_name: "serving_default"
}
inputs {
  key: "input_19"
  value {
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: 1
      }
      dim {
        size: 5
      }
      dim {
        size: 5
      }
      dim {
        size: 512
      }
    }
    tensor_content: "\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\215UCB\000\000\000\000\000\000\000\0003N\333A\000\000\000\000\000\000\000\000\000\000\000\000h\205\001@\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000T\352\022A\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\364\273\247A\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000

In [14]:
pb_response = stub.Predict(pb_request, timeout=30.0)

In [15]:
preds = pb_response.outputs['dense_15'].float_val

In [16]:
classes = ['Northern_mockingbird', 'Red_headed_Woodpecker', 'Wood_duck']

In [17]:
dict(zip(classes, preds))

{'Northern_mockingbird': 0.0, 'Red_headed_Woodpecker': 1.0, 'Wood_duck': 0.0}

### Efficient-net

In [35]:
del host, channel, stub

In [36]:
host = 'localhost:8500'

channel = grpc.insecure_channel(host)

stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

In [20]:
name = 'wood-duck.jpeg'
img = load_img(os.path.join(os.getcwd(), 'sequentials', name), target_size=(260, 260))

In [21]:
x = np.array(img)
x.shape

(260, 260, 3)

In [22]:
X = np.array([x])
X.shape

(1, 260, 260, 3)

In [23]:
X.dtype

dtype('uint8')

In [24]:
X_float = tf.cast(X, tf.float32)
X_float.dtype

tf.float32

In [37]:
pb_request = predict_pb2.PredictRequest()

pb_request.model_spec.name = 'eff-net'
pb_request.model_spec.signature_name = 'serving_default'

pb_request.inputs['input_23'].CopyFrom(np_to_protobuf(X_float))

In [38]:
pb_request

model_spec {
  name: "eff-net"
  signature_name: "serving_default"
}
inputs {
  key: "input_23"
  value {
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: 1
      }
      dim {
        size: 260
      }
      dim {
        size: 260
      }
      dim {
        size: 3
      }
    }
    tensor_content: "\000\000FC\000\000\031C\000\000\214B\000\000HC\000\000\033C\000\000\224B\000\000HC\000\000\032C\000\000\230B\000\000LC\000\000\036C\000\000\246B\000\000GC\000\000 C\000\000\246B\000\000NC\000\000$C\000\000\260B\000\000NC\000\000\"C\000\000\262B\000\000PC\000\000$C\000\000\272B\000\000OC\000\000#C\000\000\270B\000\000NC\000\000\"C\000\000\266B\000\000LC\000\000\035C\000\000\262B\000\000MC\000\000\034C\000\000\262B\000\000JC\000\000\030C\000\000\236B\000\000JC\000\000\030C\000\000\236B\000\000IC\000\000\025C\000\000\230B\000\000JC\000\000\024C\000\000\230B\000\000HC\000\000\022C\000\000\220B\000\000HC\000\000\022C\000\000\220B\000\000EC\000\000\017C\000\000\212B\000\000HC\0

In [39]:
pb_response = stub.Predict(pb_request, timeout=30.0)

In [40]:
preds = pb_response.outputs['pred'].float_val

In [41]:
dict(zip(classes, preds))

{'Northern_mockingbird': 0.13309910893440247,
 'Red_headed_Woodpecker': 0.10607355833053589,
 'Wood_duck': 0.7608273029327393}